<a href="https://colab.research.google.com/github/mathemusician/AI-Art/blob/master/Powerpoint_Maker_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
r'''
PowerPoint Maker

Made by Jv Kyle Eclarin
https://github.com/mathemusician/
'''

#@title <---- 1) Press Play to Apply Powerpoint Attributes

!pip install -q python-pptx

import ipywidgets as widgets
from IPython.display import display
import sys
from pptx import Presentation
from pptx.util import Pt
from pptx.dml.color import RGBColor
from pptx.util import Inches
from google.colab import files
from itertools import zip_longest
from math import ceil



_font_size = widgets.IntSlider(
    value=50,
    min=30,
    max=80,
    step=1,
    description='Font Size:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    indent=True,
    font_size=30
)

_split_lyrics_by = widgets.Dropdown(
    options=['1', '2', '3', '4'],
    value='2',
    description='Split Lyrics:',
    disabled=False,
)

_powerpoint_name = widgets.Text(
    value='Powerpoint Name',
    placeholder='Type something',
    description='Name:',
    disabled=False
)

_lyrics_and_sections = widgets.Textarea(
    value='Example Lyrics\nYay!',
    placeholder='Type something',
    description='Lyrics:',
    disabled=False
)


button = widgets.Button(description="Download Lyrics",
                        indent=True)

output = widgets.Output()



display(_font_size)
display(_split_lyrics_by)
display(_powerpoint_name)
display(_lyrics_and_sections)
display(button, output)





def grouper(n, iterable, padvalue=''):
  "grouper(3, 'abcdefg', 'x') --> ('a','b','c'), ('d','e','f'), ('g','x','x')"
  return zip_longest(*[iter(iterable)]*n, fillvalue=padvalue)



def split_lyrics_by_number(lyrics_and_sections, split_number):
  
  groups = grouper(int(split_number), lyrics_and_sections)
  num_iters = ceil( len(lyrics_and_sections) / int(split_number) )
  lyrics_and_sections = ['\n'.join(next(groups)) for i in range(num_iters)]
  
  # get rid of trailing newlines
  lyrics_and_sections[-1] = lyrics_and_sections[-1].strip()
  
  return lyrics_and_sections



def split_by_font_size(lyrics_and_sections):
  
  parsed_lyrics = []

  for lyric in lyrics_and_sections:

      if len(lyric) > 50:
          # just split it in half
          lyric_list = lyric.split(' ')
          length_of_lyric = len(lyric_list)

          if '\n' in lyric:
              length_of_first_part = lyric.index('\n')
              lyric = lyric.replace('\n','')
              parsed_lyrics.append(lyric[:length_of_first_part])
              parsed_lyrics.append(lyric[length_of_first_part:])

          else:
              length_of_first_part = int(length_of_lyric/2)
              first_part = lyric_list[:length_of_first_part]
              second_part = lyric_list[length_of_first_part:]
              parsed_lyrics.append(' '.join(first_part))
              parsed_lyrics.append(' '.join(second_part))

      else:
          parsed_lyrics.append(lyric)
  
  return parsed_lyrics



def get_rid_of_informative_lyrics(lyrics_and_sections):
  
  parsed_lyrics = []
  
  for lyric in lyrics_and_sections:
    if ('[' in lyric) or (']' in lyric):
      pass
    else:
      parsed_lyrics.append(lyric)
  
  return parsed_lyrics


def make_powerpoint(font_size, split_lyrics_by, powerpoint_name, lyrics_and_sections):

  # split lyrics by ones
  lyrics_and_sections = lyrics_and_sections.split('\n')

  # get rid of informative lyrics
  lyrics_and_sections = get_rid_of_informative_lyrics(lyrics_and_sections)

  # split lyrics using font sizes
  lyrics_and_sections = split_by_font_size(lyrics_and_sections)

  # split lyrics by number
  lyrics_and_sections = split_lyrics_by_number(lyrics_and_sections, split_lyrics_by)


  # Make presentation
  prs = Presentation()
  title_slide_layout = prs.slide_layouts[0]

  # make slides
  for lyric in lyrics_and_sections:
   
      slide = prs.slides.add_slide(title_slide_layout)
      title = slide.shapes.title
      title.text = lyric
      
      # make background black
      fill = slide.background.fill
      fill.solid()
      fill.fore_color.theme_color = 1
      
      # Center text
      if split_lyrics_by == '1':
        title.top = Inches(3)
      elif split_lyrics_by == '2':
        title.top = Inches(3)
      elif split_lyrics_by == '3':
        title.top = Inches(2.5)
      elif split_lyrics_by == '4':
        title.top = Inches(2)
      title.left = Inches(0)
      title.width = Inches(10)
      title.height = 1470025
      
      # format individual lines of words
      for para_id in range(len(slide.shapes.title.text_frame.paragraphs)):
          title.text_frame.paragraphs[para_id].font.bold = True
          title.text_frame.paragraphs[para_id].font.name = 'Helvetica'
          title.text_frame.paragraphs[para_id].font.size = Pt(font_size)  # can be 58 or 44
          title.text_frame.paragraphs[para_id].font.color.rgb = RGBColor(255, 255, 255)


  prs.save(f'{powerpoint_name}.pptx')
  files.download(f'{powerpoint_name}.pptx')

def on_button_clicked(b):
  with output:
    font_size = _font_size.value
    split_lyrics_by = _split_lyrics_by.value
    powerpoint_name = _powerpoint_name.value
    lyrics_and_sections = _lyrics_and_sections.value

    make_powerpoint(font_size, split_lyrics_by, powerpoint_name, lyrics_and_sections)

button.on_click(on_button_clicked)

IntSlider(value=50, continuous_update=False, description='Font Size:', max=80, min=30)

Dropdown(description='Split Lyrics:', index=1, options=('1', '2', '3', '4'), value='2')

Text(value='Powerpoint Name', description='Name:', placeholder='Type something')

Textarea(value='Example Lyrics\nYay!', description='Lyrics:', placeholder='Type something')

Button(description='Download Lyrics', style=ButtonStyle())

Output()